In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from astropy.extern.six.moves.urllib import request
import tarfile

from spect_utils import read_spec

%matplotlib inline

In [ ]:
url = 'http://python4astronomers.github.io/_downloads/astropy_UVES.tar.gz'
tarfile.open(fileobj=request.urlopen(url), mode='r|*').extractall()

In [3]:
from glob import glob

import numpy as np

from astropy.wcs import WCS
from astropy.io import fits

# glob searches through directories similar to the Unix shell
filelist = glob('UVES/*.fits')
# sort alphabetically - given the way the filenames are
# this also sorts in time
filelist.sort()

In [4]:
sp = fits.open(filelist[0])
sp.info()

Filename: UVES/r.UVES.2011-08-11T232352.266-A01_0000.fits
No.    Name         Type      Cards   Dimensions   Format
  0  PRIMARY     PrimaryHDU     611   (42751,)   float32   


In [5]:
header = sp[0].header

wcs = WCS(header)
#make index array
index = np.arange(header['NAXIS1'])

wavelength = wcs.wcs_pix2world(index[:,np.newaxis], 0)
wavelength.shape
#Ahh, this has the wrong dimension. So we flatten it.
wavelength = wavelength.flatten()

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


In [6]:
flux = sp[0].data

In [7]:
flux = np.zeros((len(filelist), len(wavelength)))
# date comes as string with 23 characters (dtype = 'S23')
date = np.zeros((len(filelist)), dtype = 'S23')

for i, fname in enumerate(filelist):
    w, f, date_obs = read_spec(fname)
    flux[i,:] = f
    date[i] = date_obs

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


In [8]:
import astropy.units as u
from astropy.constants.si import c, G, M_sun, R_sun

wavelength = wavelength * u.AA

# Let's define some constants we need for the exercises further down
# Again, we multiply the value with a unit here
heliocentric = -23. * u.km/u.s
v_rad = -4.77 * u.km / u.s  # Strassmeier et al. (2005)
R_MN_Lup = 0.9 * R_sun      # Strassmeier et al. (2005)
M_MN_Lup = 0.6 * M_sun      # Strassmeier et al. (2005)
vsini = 74.6 * u.km / u.s   # Strassmeier et al. (2005)
period = 0.439 * u.day      # Strassmeier et al. (2005)

inclination = 45. * u.degree # Strassmeier et al. (2005)
# All numpy trigonometric functions expect the input in radian.
# So far, astropy does not know this, so we need to convert the
# angle manually
incl = inclination.to(u.radian)

In [9]:
v_accr = (2.* G * M_MN_Lup/R_MN_Lup)**0.5
v_rot = vsini / np.sin(incl)
wavelength = wavelength * (1. * u.dimensionless_unscaled+ heliocentric/c)

In [10]:
from astropy.time import Time
t1 = Time(header['MJD-Obs'], format = 'mjd', scale = 'utc')
t2 = Time(header['Date-Obs'], scale = 'utc')